# 1- Merging genotyping datasets (*Immuno* and *NeuroX*)

In [1]:
from scipy.stats import norm
from matplotlib_venn import venn3

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import subprocess
import math
import sys
import os

## Introduction

During this step of the preprocessing phase of the workflow, we will merge the *Immuno* and *NeuroX* genotyping datasets in a single one.

Genetic variant positions in *Immuno* datasets were mapped on hg18 genome assembly, while *NeuroX* SNPs were mapped on hg19 assembly. This could cause issues and inconsistencies when merging together the two set of genomic variants. To solve this issue we must lift *Immuno* variants positions from hg18 to corresponding hg19 genomic coordinates. 

After *Immuno* SNP positions lifting from hg18 to hg19, we merge the two datasets. During the following steps of the workflow, will be used the merged genetic variants dataset.

### Exploring *Immuno* and *NeuroX*

Each dataset have 3 files:

- **BIM**: contains SNPs, their genomic position (chromosome and position) and their genotype status (heterozygote or homozygous)
    
- **FAM**: contains information related to the subjects (sex, ID, etc.)
    
- **BED**: binary file representing a matrix with subjects ID on the rows and SNP IDs on the columns

In [2]:
immuno_path = "../../data/genotyping/IMMUNO/"
neurox_path = "../../data/genotyping/NEUROX/"

Explore raw genotyping data contained in the two BIMs.

Let's start with *Immuno*.

In [3]:
immuno_bim_fn = os.path.join(immuno_path, "IMMUNO.bim")
neurox_bim_fn = os.path.join(neurox_path, "NEUROX.bim")

immuno_bim = pd.read_csv(immuno_bim_fn, sep="\t", header=None)
neurox_bim = pd.read_csv(neurox_bim_fn, sep="\t", header=None)

In [4]:
immuno_bim.head(n=10)

,0,1,2,3,4,5
0,1,imm_1_898835,0,898835,0,A
1,1,vh_1_1108138,0,1108138,T,C
2,1,vh_1_1110294,0,1110294,A,G
3,1,rs9729550,0,1125105,C,A
4,1,rs1815606,0,1130298,T,G
5,1,rs7515488,0,1153667,T,C
6,1,rs11260562,0,1155173,A,G
7,1,rs6697886,0,1163474,A,G
8,1,1_1168711,0,1168711,A,G
9,1,rs6603785,0,1176365,T,A


And let's look at *NeuroX*.

In [5]:
neurox_bim.head(n=10)

,0,1,2,3,4,5
0,1,NeuroX_PARK7_Pro158del,0,0,0,I
1,1,NeuroX_PINK1_23bp_del_ex7,0,0,0,I
2,1,NeuroX_PINK1_534_535insQ,0,0,0,D
3,1,NeuroX_PINK1_Asp525fs,0,0,0,D
4,1,NeuroX_PINK1_Cys549fs,0,0,0,I
5,1,NeuroX_PINK1_Lys520fs,0,0,0,I
6,1,exm2268640,0,762320,T,C
7,1,exm41,0,861349,0,C
8,1,exm1916089,0,865545,0,0
9,1,exm44,0,865584,0,G


## Lifting *Immuno* SNP positions to hg19 genome assembly

As mentioned above, the *Immuno* genetic variants positions are mapped on hg18, while those of *NeuroX* are mapped on hg19 genome assembly. To solve this problem we lift *Immuno* SNP positions to the corresponding hg19 genomic locations. 

To lift SNP positions we use the web-based tool liftOver at the UCSC web-server (https://genome.ucsc.edu/cgi-bin/hgLiftOver).

Since liftOver accepts as input UCSC BED files and not BIMs, we first convert *Immuno* BIM file in the corresponding UCSC BED file.

In [6]:
# minimal UCSC BED file requires
#
#CHROM    START    STOP    FEATURE_ID 

tmp_chr = immuno_bim.iloc[:,0]  # chromosomes
tmp_pos = immuno_bim.iloc[:,3]  # SNP positions
tmp_name = immuno_bim.iloc[:,1] # SNP IDs

# append 'chr' in front of chromosome numbers (required by liftOver)
tmp_chr_str = [''.join(["chr", str(c)]) for c in tmp_chr.values]

# replace chr23, chr24, chr25 and chr26 with chrX, chrY, chrX and chrMT
tmp_chr_str2 = [c.replace('23', 'X') for c in tmp_chr_str]
tmp_chr_str3 = [c.replace('24', 'Y') for c in tmp_chr_str2]
tmp_chr_str4 = [c.replace('25', 'X') for c in tmp_chr_str3]
tmp_chr_str5 = [c.replace('26', 'MT') for c in tmp_chr_str4]

# write the resulting UCSC BED file
bed = pd.concat(
    [pd.DataFrame(tmp_chr_str5), tmp_pos, tmp_pos+1, tmp_name],
    axis=1
)
print(bed.head(n=5))
print(bed.tail(n=5))
bed.to_csv(
    os.path.join(immuno_path, "IMMUNO_tolift.bed"), sep="\t",
    index=False, header=False
)

      0        3        3             1
0  chr1   898835   898836  imm_1_898835
1  chr1  1108138  1108139  vh_1_1108138
2  chr1  1110294  1110295  vh_1_1110294
3  chr1  1125105  1125106     rs9729550
4  chr1  1130298  1130299     rs1815606
            0          3          3           1
196519   chrX  154889906  154889907   rs3093525
196520   chrX  154889917  154889918   rs3093526
196521   chrX  154892630  154892631   rs2742301
196522   chrX  154907376  154907377   rs2981835
196523  chrMT       3721       3722  MitoA3721G


Now we can map *Immuno* positions to hg19 genome assembly using liftOver (https://genome.ucsc.edu/cgi-bin/hgLiftOver)

LiftOver mapped succesfully 196,520 of IMMUNO SNPs of the original 196,524 genetic variants.

The four unmapped SNPs were:
- imm_3_50875337
- rs1574660
- rs1131012
- MitoA3721G

Before proceeding, we rename the resulting UCSC BED file with the lifted SNPs as ```IMMUNO_lifted.bed```.

In [7]:
bed_lifted = pd.read_csv(os.path.join(immuno_path, "IMMUNO_lifted.bed"), sep="\t", header=None)

# we take SNPs whose conversion to hg19 positions was succesful
good_snps = bed_lifted.iloc[:,3]
good_snps.to_csv(
    os.path.join(immuno_path, "good_snps.txt"),
    index=False, header=False
)

We keep the mapped SNPs and we create the corresponding BIM, FAM and BED files (named ```IMMUNO_hg19```).

In [8]:
!plink --bfile {os.path.join(immuno_path, "IMMUNO")} --update-map {os.path.join(immuno_path, "IMMUNO_lifted.bed")} 2 4 --make-bed --extract {os.path.join(immuno_path, "good_snps.txt")} --out {os.path.join(immuno_path, "IMMUNO_hg19")}

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ../../data/genotyping/IMMUNO/IMMUNO_hg19.log.
Options in effect:
  --bfile ../../data/genotyping/IMMUNO/IMMUNO
  --extract ../../data/genotyping/IMMUNO/good_snps.txt
  --make-bed
  --out ../../data/genotyping/IMMUNO/IMMUNO_hg19
  --update-map ../../data/genotyping/IMMUNO/IMMUNO_lifted.bed 2 4

16384 MB RAM detected; reserving 8192 MB for main workspace.
196524 variants loaded from .bim file.
523 people (343 males, 180 females) loaded from .fam.
--update-map: 196520 values updated.
--extract: 196520 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 523 founders and 0 nonfounders present.
Calculating allele frequencies... 1011121314151617181920212223242526272829303132333435363738394041424344454647484950515253545556575859606162636465666768697071727374757677787980818283

## Merging *Immuno* and *NeuroX*

Since both *Immuno* and *NeuroX* SNP positions are now mapped on the same genome assembly (hg19) we can merge the two datasets.

In [9]:
immuno_bim = pd.read_csv(os.path.join(immuno_path, "IMMUNO_hg19.bim"), sep="\t", header=None)  # BIM with SNPs mapped on hg19
immuno_bim.head(n=10)

,0,1,2,3,4,5
0,1,imm_1_898835,0,908972,0,A
1,1,vh_1_1108138,0,1118275,T,C
2,1,vh_1_1110294,0,1120431,A,G
3,1,rs9729550,0,1135242,C,A
4,1,rs1815606,0,1140435,T,G
5,1,rs7515488,0,1163804,T,C
6,1,rs11260562,0,1165310,A,G
7,1,rs6697886,0,1173611,A,G
8,1,1_1168711,0,1178848,A,G
9,1,rs6603785,0,1186502,T,A


In [10]:
neurox_bim.head(n=10)  # look again at NeuroX

,0,1,2,3,4,5
0,1,NeuroX_PARK7_Pro158del,0,0,0,I
1,1,NeuroX_PINK1_23bp_del_ex7,0,0,0,I
2,1,NeuroX_PINK1_534_535insQ,0,0,0,D
3,1,NeuroX_PINK1_Asp525fs,0,0,0,D
4,1,NeuroX_PINK1_Cys549fs,0,0,0,I
5,1,NeuroX_PINK1_Lys520fs,0,0,0,I
6,1,exm2268640,0,762320,T,C
7,1,exm41,0,861349,0,C
8,1,exm1916089,0,865545,0,0
9,1,exm44,0,865584,0,G


Before merging *Immuno* and *NeuroX*, we keep only those subjects with genotyping data available in both the considered datasets.

In [11]:
immuno_fam_fn = os.path.join(immuno_path, "IMMUNO_hg19.fam")
neurox_fam_fn = os.path.join(neurox_path, "NEUROX.fam")

immuno_fam = pd.read_csv(immuno_fam_fn, sep=" ", header=None)
neurox_fam = pd.read_csv(neurox_fam_fn, sep=" ", header=None)

Let's explore the two FAM files content.

Let's begin with *Immuno*.

In [12]:
immuno_fam.head(n=5)

,0,1,2,3,4,5
0,3400,3400,0,0,2,-9
1,3401,3401,0,0,2,-9
2,3402,3402,0,0,1,-9
3,3403,3403,0,0,1,-9
4,3404,3404,0,0,2,-9


And *NeuroX*.

In [13]:
neurox_fam.head(n=5)

,0,1,2,3,4,5
0,3527,3527,0,0,1,-9
1,3274,3274,0,0,1,-9
2,3220,3220,0,0,2,-9
3,3467,3467,0,0,1,-9
4,3171,3171,0,0,1,-9


And we retrieve the FAMID and IIDs of those subjects with genotyping data in both *Immuno* and *NeuroX*. 

In [14]:
immuno_subj = immuno_fam.iloc[:,1].tolist()
neurox_subj = neurox_fam.iloc[:,1].tolist()

# get the subjects appearing in both IMMUNO and NEUROX data
common_subj = set(immuno_subj).intersection(set(neurox_subj))

# write the common subjects to a file
with open("../../data/genotyping/common_subj.txt", mode="w+") as outfile:
    for s in common_subj:
        outfile.write(''.join([str(s), " ", str(s), "\n"]))
        
common_subj_fn = "../../data/genotyping/common_subj.txt"

Originally, in *Immuno* there are 523 subjects, while in *NeuroX* there are 619 subjects. 

The subjects whose genotyping data are available in both the datasets are 520. 

Before merging *Immuno* and *NeuroX*, we keep data belonging only to the subjects shared by the two considered datsets.

In [15]:
!plink --bfile {os.path.join(immuno_path, "IMMUNO_hg19")} --keep {common_subj_fn} --make-bed --out {os.path.join(immuno_path, "IMMUNO_common")}

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ../../data/genotyping/IMMUNO/IMMUNO_common.log.
Options in effect:
  --bfile ../../data/genotyping/IMMUNO/IMMUNO_hg19
  --keep ../../data/genotyping/common_subj.txt
  --make-bed
  --out ../../data/genotyping/IMMUNO/IMMUNO_common

16384 MB RAM detected; reserving 8192 MB for main workspace.
196520 variants loaded from .bim file.
523 people (343 males, 180 females) loaded from .fam.
--keep: 520 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 520 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
../../data/genotyping/IMMUNO/IMMUNO_common.hh ); many commands treat the

In [16]:
!plink --bfile {os.path.join(neurox_path, "NEUROX")} --keep {common_subj_fn} --make-bed --out {os.path.join(neurox_path, "NEUROX_common")}

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ../../data/genotyping/NEUROX/NEUROX_common.log.
Options in effect:
  --bfile ../../data/genotyping/NEUROX/NEUROX
  --keep ../../data/genotyping/common_subj.txt
  --make-bed
  --out ../../data/genotyping/NEUROX/NEUROX_common

16384 MB RAM detected; reserving 8192 MB for main workspace.
267607 variants loaded from .bim file.
619 people (409 males, 210 females) loaded from .fam.
--keep: 520 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 520 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
../../data/genotyping/NEUROX/NEUROX_common.hh ); many commands treat these as

PPMI documentation tells us that the genotyping was made using custom arrays. Thus, the SNP names do not always are identified with standard rsIDs. Moreover, we want to keep as much SNPs as possible from the two datasets.

To get the SNPs which are uniquely in *Immuno*, we need to work on their genomic coordinates.

In [17]:
#function to identify snps by chrom_pos_pos+1
def rename_SNP(snpname, chrom, pos, a1, a2):
    if chrom == 0 or pos == 0:
        return snpname  # manage unmapped SNPs on neurox
    return "_".join([str(chrom), str(pos), str(pos+1), str(a1), str(a2)])

# the BIM are those containing the common subjects data
immuno_bim = pd.read_csv(
    os.path.join(immuno_path, "IMMUNO_common.bim"), sep="\t",
    header=None
)
neurox_bim = pd.read_csv(
    os.path.join(neurox_path, "NEUROX_common.bim"), sep="\t",
    header=None
)

renamedsnps_immuno = immuno_bim.apply(lambda row : rename_SNP(row[1], row[0], row[3], row[4], row[5]), axis=1)
immuno_coords = immuno_bim
immuno_coords[6] = renamedsnps_immuno

renamedsnps_neurox = neurox_bim.apply(lambda row : rename_SNP(row[1], row[0], row[3], row[4], row[5]), axis=1)
neurox_coords = neurox_bim
neurox_coords[6] = renamedsnps_neurox

In [18]:
immuno_coords.head(n=10)

,0,1,2,3,4,5,6
0,1,imm_1_898835,0,908972,0,A,1_908972_908973_0_A
1,1,vh_1_1108138,0,1118275,T,C,1_1118275_1118276_T_C
2,1,vh_1_1110294,0,1120431,A,G,1_1120431_1120432_A_G
3,1,rs9729550,0,1135242,C,A,1_1135242_1135243_C_A
4,1,rs1815606,0,1140435,T,G,1_1140435_1140436_T_G
5,1,rs7515488,0,1163804,T,C,1_1163804_1163805_T_C
6,1,rs11260562,0,1165310,A,G,1_1165310_1165311_A_G
7,1,rs6697886,0,1173611,A,G,1_1173611_1173612_A_G
8,1,1_1168711,0,1178848,A,G,1_1178848_1178849_A_G
9,1,rs6603785,0,1186502,T,A,1_1186502_1186503_T_A


In [19]:
neurox_coords.head(n=10)

,0,1,2,3,4,5,6
0,1,NeuroX_PARK7_Pro158del,0,0,0,I,NeuroX_PARK7_Pro158del
1,1,NeuroX_PINK1_23bp_del_ex7,0,0,0,I,NeuroX_PINK1_23bp_del_ex7
2,1,NeuroX_PINK1_534_535insQ,0,0,0,D,NeuroX_PINK1_534_535insQ
3,1,NeuroX_PINK1_Asp525fs,0,0,0,D,NeuroX_PINK1_Asp525fs
4,1,NeuroX_PINK1_Cys549fs,0,0,0,I,NeuroX_PINK1_Cys549fs
5,1,NeuroX_PINK1_Lys520fs,0,0,0,I,NeuroX_PINK1_Lys520fs
6,1,exm2268640,0,762320,T,C,1_762320_762321_T_C
7,1,exm41,0,861349,0,C,1_861349_861350_0_C
8,1,exm1916089,0,865545,0,0,1_865545_865546_0_0
9,1,exm44,0,865584,0,G,1_865584_865585_0_G


In [20]:
# we get the names of the SNPs only available in IMMUNO dataset
# and write the SNPs uniquely appearing in IMMUNO to a TXT file
only_immuno_snps_fn = "../../data/genotyping/onlyIMMUNOsnps.txt"
neurox_coords_set = set(neurox_coords.iloc[:,6].to_list())
immuno_coords[~immuno_coords[6].isin(neurox_coords_set)].iloc[:,1].to_csv(only_immuno_snps_fn, header=False, index=False)

Then, in *Immuno* we keep only those SNPs which are not available in *NeuroX* dataset.

In [21]:
!plink --bfile {os.path.join(immuno_path, "IMMUNO_common")} --extract {only_immuno_snps_fn} --make-bed --out {os.path.join(immuno_path, "IMMUNO_uniquesnps")}

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ../../data/genotyping/IMMUNO/IMMUNO_uniquesnps.log.
Options in effect:
  --bfile ../../data/genotyping/IMMUNO/IMMUNO_common
  --extract ../../data/genotyping/onlyIMMUNOsnps.txt
  --make-bed
  --out ../../data/genotyping/IMMUNO/IMMUNO_uniquesnps

16384 MB RAM detected; reserving 8192 MB for main workspace.
196520 variants loaded from .bim file.
520 people (341 males, 179 females) loaded from .fam.
--extract: 189564 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 520 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
../../data/genotyping/IMMUNO/IMMUNO_uniquesnps.h

We can now merge *Immuno* and *NeuroX* in a single datset.

Note that are created also the corresponding BIM, FAM and BED, representing the merged data.

In [22]:
merged_ppmi_fn = "../../data/genotyping/PPMI_merge"

!plink --bfile {os.path.join(neurox_path, "NEUROX_common")} --bmerge {os.path.join(immuno_path, "IMMUNO_uniquesnps")} --make-bed --out {merged_ppmi_fn}

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ../../data/genotyping/PPMI_merge.log.
Options in effect:
  --bfile ../../data/genotyping/NEUROX/NEUROX_common
  --bmerge ../../data/genotyping/IMMUNO/IMMUNO_uniquesnps
  --make-bed
  --out ../../data/genotyping/PPMI_merge

16384 MB RAM detected; reserving 8192 MB for main workspace.
520 people loaded from ../../data/genotyping/NEUROX/NEUROX_common.fam.
520 people to be merged from
../../data/genotyping/IMMUNO/IMMUNO_uniquesnps.fam.
Of these, 0 are new, while 520 are present in the base dataset.
267607 markers loaded from ../../data/genotyping/NEUROX/NEUROX_common.bim.
189564 markers to be merged from
../../data/genotyping/IMMUNO/IMMUNO_uniquesnps.bim.
Of these, 189564 are new, while 0 are present in the base dataset.
the same position.
have the same position.
the same position.
3325 more same-position warnings: see log 